In [1]:
class NodeTree(object):
    def __init__(self):
        self.left = None
        self.right = None
        self.parent = None


class BinaryTree(object):
    def __init__(self):
        super(BinaryTree, self).__init__()
        self.nil = None
        self.root = None
        self.initialize()

    def initialize(self):
        self.root = None

    def node_depth(self, node: NodeTree):
        """
        сколько нод надо пройти до рута
        """
        tree_depth = 0
        while node != self.root:
            node = node.parent
            tree_depth += 1
        return tree_depth

    def node_size(self, node: NodeTree):
        """
        количество нод расположенных ниже,
        включая переданную ноду
        """
        if node == self.nil:
            return 0
        else:
            left_size = self.node_size(node.left)
            right_size = self.node_size(node.right)
            return 1 + right_size + left_size

    def node_height(self, node: NodeTree):
        if node == self.nil:
            return 0
        else:
            left_size = self.node_size(node.left)
            right_size = self.node_size(node.right)
            return 1 + max(right_size, left_size)

    def traverse_recursive(self, node: NodeTree):
        if node != self.nil:
            self.traverse_recursive(node.left)
            self.traverse_recursive(node.right)

    def traverse_iterative(self):
        node: NodeTree = self.root
        prev_node: NodeTree = self.nil
        next_node: NodeTree = self.nil

        while node != self.nil:
            # если прошлая нода не является родителем
            if prev_node == node.parent:
                # если нода слева не равна null тогда
                # следующая нода будет left
                if node.left != self.nil:
                    next_node = node.left
                # иначе правая
                elif node.right != self.nil:
                    next_node = node.right
                else:
                    # если нода не имеет потомков, тогда
                    # следующей нодой будет родитель
                    next_node = node.parent
            # если предыдущая нода это левая текущей
            elif prev_node == node.left:
                # если правая текущей не пусто,
                # значит она будет следующая
                if node.right != self.nil:
                    next_node = node.right
                # если пустая, значит следующая будет родителем
                else:
                    next_node = node.parent
            else:
                # если все
                next_node = node.parent

            prev_node = node
            node = next_node

Depth: 1
Size: 3
Size2: 0
Height: 2
